## Example 5: Extractive Distillation

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from thermo_models.RaoultsLawModel import *
from distillation.DistillationModel import DistillationModel
import utils.AntoineEquation as AE
from IPython.display import clear_output
from tqdm import tqdm
from distillation.DistillationDoubleFeed import *
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interactive, Button, VBox, HBox, Output
from IPython.display import clear_output
from thermo_models.VanLaarModel import *
from distillation.residue_curves import *

In [2]:
#Acetone (1 - Light) Methanol (2 - Intermediate) Water (3 - Heavy)
#Table A.6 in Knapp 
A_ij = {
    (1,1):0,
    (1,2):182.0,
    (1,3):795.0,
    (2,1):196,
    (2,2):0,
    (2,3):332.6,
    (3,1):490.0,
    (3,2):163.80,
    (3,3):0
}

#Different definition of Antoine where we have to take the negative of B
Acet_A = 21.3099; Acet_B = 2801.53; Acet_C = -42.875
Meth_A = 23.4832; Meth_B = 3634.01; Meth_C = -33.768
#Assuming P < 2 atm
Water_A = 23.2256; Water_B = 3835.18; Water_C = -45.343

#Kanapp Thesis Figure 3.8 uses ln form of Antoine
AcetoneAntoine = AE.AntoineEquationBaseE(Acet_A,Acet_B,Acet_C)
MethanolAntoine = AE.AntoineEquationBaseE(Meth_A, Meth_B, Meth_C)
WaterAntoine = AE.AntoineEquationBaseE(Water_A,Water_B,Water_C)
        
P_sys = 1.0325
# Create a Raoult's law object
AceMethWaterSys = VanLaarModel(3, P_sys = P_sys, A_coeff = A_ij, comp_names = ["Ace","Meth","Water"], partial_pressure_eqs = [AcetoneAntoine, MethanolAntoine, WaterAntoine], A=None, B=None)


In [3]:
out_residue = Output()
rcm = residue_curve(AceMethWaterSys)
fig, ax = plt.subplots(1,1,figsize= (7,7))
rcm.plot_residue_curve_int(ax,15)

convxtoy Runs: 500 fvec: [-0.2539455  -0.25392391 -0.25392353 -0.26218602]


c:\Users\kelle\newCTEI\ctei_ext_distillation\src\utils\AntoineEquation.py:89: RuntimeWarning: overflow encountered in exp
  return np.exp((self.A - self.B/(Temp + self.C)))


convxtoy Runs: 500 fvec: [-0.25379769 -0.25524288 -0.25344353 -0.26151661]


c:\Users\kelle\newCTEI\ctei_ext_distillation\src\thermo_models\VLEModelBaseClass.py:165: RuntimeWarning: overflow encountered in multiply
  lefths = x_array * self.get_activity_coefficient(x_array, Temp=Temp) * self.get_vapor_pressure(Temp)


convxtoy Runs: 500 fvec: [-0.25517724 -0.25294665 -0.25281283 -0.26301529]
convxtoy Runs: 500 fvec: [-0.25435722 -0.25350169 -0.2535644  -0.26254401]
convxtoy Runs: 500 fvec: [-0.25393163 -0.25393152 -0.25393153 -0.26218434]
convxtoy Runs: 500 fvec: [-0.25463881 -0.25380188 -0.25382744 -0.26172578]
convxtoy Runs: 500 fvec: [-0.25399406 -0.25389177 -0.25389933 -0.26219355]
convxtoy Runs: 500 fvec: [-0.25472037 -0.25333064 -0.25315005 -0.26275926]
convxtoy Runs: 500 fvec: [-0.25470685 -0.2537618  -0.25380116 -0.26172415]
convxtoy Runs: 500 fvec: [-0.25455288 -0.25337767 -0.25339736 -0.26263641]
convxtoy Runs: 500 fvec: [-0.25422587 -0.2537352  -0.25374504 -0.26227011]
convxtoy Runs: 500 fvec: [-0.25399781 -0.25393366 -0.25393447 -0.26211532]
convxtoy Runs: 500 fvec: [-0.25467867 -0.25366656 -0.25367009 -0.26197065]
convxtoy Runs: 500 fvec: [-0.25408649 -0.25383474 -0.25384219 -0.26221461]
convxtoy Runs: 500 fvec: [-0.25437277 -0.25352407 -0.25355181 -0.26251947]
convxtoy Runs: 500 fvec: 